### Notebook to check the dates of the MRI scans

#### Date Sources
1. Suivi sheet 
2. Testing sheet
3. DICOM file 
4. DICOM header

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
DATASET_ROOT = "/home/nikhil/projects/Parkinsons/qpn/"

# Current nipoppy release
current_release = "Aug_2024"

data_release_dir = f"{DATASET_ROOT}/releases/{current_release}/"
tabular_data_release_dir = f"{data_release_dir}/tabular/"

current_manifest_csv = f"{data_release_dir}/manifest.csv"
MRI_acq_dates_Aug_csv = f"{tabular_data_release_dir}/recruitment/MRI_acqdata.csv"
MRI_acq_dates_Sept_csv = f"{tabular_data_release_dir}/recruitment/MRI_acqdata_Sept2024.csv"



### Read current manifest

In [3]:
manifest_df= pd.read_csv(current_manifest_csv)

manifest_participants = manifest_df["participant_id"].unique()
n_manifest_participants = len(manifest_participants)
print(f"Number of participants in manifest: {n_manifest_participants}")

sessions = manifest_df["session"].unique()
print(f"sessions: {sessions}")

per_session_counts = manifest_df.value_counts("session")
print(f"per_session_counts: {per_session_counts}")

manifest_df.head()

Number of participants in manifest: 306
sessions: ['ses-01' 'ses-02' 'ses-03']
per_session_counts: session
ses-01    306
ses-02     68
ses-03      5
Name: count, dtype: int64


,participant_id,visit,session,datatype,dicom_availabilty
0,MNI0215,MRI_v1,ses-01,"['anat','dwi','fmap','func']",yes
1,PD01145,MRI_v1,ses-01,"['anat','dwi','fmap','func']",yes
2,MNI0313,MRI_v1,ses-01,"['anat','dwi','fmap','func']",yes
3,MNI0476,MRI_v1,ses-01,"['anat','dwi','fmap','func']",yes
4,PD01214,MRI_v1,ses-01,"['anat','dwi','fmap','func']",yes


### MRI dates

In [4]:
mri_acq_dates_aug_df = pd.read_csv(MRI_acq_dates_Aug_csv)
mri_acq_dates_sept_df = pd.read_csv(MRI_acq_dates_Sept_csv, index_col=0)

mri_acq_dates_aug_df.head()

,participant_id,session,participant_dicom_dir,scanner_acq_date,Confirmed existance,DVD Confirmed?,DICOM server,LORIS,Notes,Visual QC,rater
0,MNI0028,ses-01,MNI0028_152209_MRI01_MNI_20231004_152124681,2023-10-04,Yes,Yes,NaN,NaN,NaN,NaN,NaN
1,MNI0056,ses-01,MNI0056_864854_MRI01_MNI_20210818_151510608,2021-08-18,Yes,Yes,NaN,NaN,NaN,Artifact in the right hemisphere in T1w,NB
2,MNI0058,ses-01,MNI0058_197308_MRI01_MNI_20210818_105219098,2021-08-18,Yes,Yes,NaN,NaN,NaN,NaN,NaN
3,MNI0068,ses-01,MNI0068_842090_MRI01_MNI_20210827_150412426,2021-08-27,Yes,Yes,NaN,NaN,NaN,NaN,NaN
4,MNI0079,ses-01,MNI0079_760662_MRI01_MNI_20211222_134225878,2021-12-22,Yes,Yes,NaN,NaN,NaN,NaN,NaN


In [5]:
mri_acq_dates_sept_df = mri_acq_dates_sept_df.rename(columns={"scanner_acq_date": "scanner_acq_date_sept",
                                                              "participant_dicom_dir": "participant_dicom_dir_sept"})
mri_acq_dates_sept_df.head()

,participant_id,session,participant_dicom_dir_sept,scanner_acq_date_sept
index,,,,
0,MNI0028,ses-01,MNI0028_152209_MRI01_MNI_20231004_152124681,2023-10-04
1,MNI0056,ses-01,MNI0056_864854_MRI01_MNI_20210818_151510608,2021-08-18
2,MNI0058,ses-01,MNI0058_197308_MRI01_MNI_20210818_105219098,2021-08-18
3,MNI0068,ses-01,MNI0068_842090_MRI01_MNI_20210827_150412426,2021-08-27
4,MNI0079,ses-01,MNI0079_760662_MRI01_MNI_20211222_134225878,2021-12-22


In [6]:
mri_acq_dates_sept_df[mri_acq_dates_sept_df["participant_id"] == "MNI0369"]

,participant_id,session,participant_dicom_dir_sept,scanner_acq_date_sept
index,,,,
60,MNI0369,ses-01,MNI0369,NaN
61,MNI0369,ses-02,MNI0369,NaN


### Merge and compare dicoms and dates

In [7]:
mri_acq_dates_aug_df["scanner_acq_date"] = pd.to_datetime(mri_acq_dates_aug_df["scanner_acq_date"])
mri_acq_dates_sept_df["scanner_acq_date_sept"] = pd.to_datetime(mri_acq_dates_sept_df["scanner_acq_date_sept"])

# merge the two dataframes
mri_acq_dates_aug_df = mri_acq_dates_aug_df[["participant_id", "session", "scanner_acq_date", "participant_dicom_dir"]]
mri_acq_dates_df = pd.merge(mri_acq_dates_aug_df, mri_acq_dates_sept_df, on=["participant_id","session"], how="left")


mri_acq_dates_df["dicom_dir_check"] = mri_acq_dates_df["participant_dicom_dir"] == mri_acq_dates_df["participant_dicom_dir_sept"]
mri_acq_dates_df["acq_date_check"] = mri_acq_dates_df["scanner_acq_date"] == mri_acq_dates_df["scanner_acq_date_sept"]

# Get date differences
mri_acq_dates_df["date_diff"] = mri_acq_dates_df["scanner_acq_date"] - mri_acq_dates_df["scanner_acq_date_sept"]

mri_acq_dates_df.head()

,participant_id,session,scanner_acq_date,participant_dicom_dir,participant_dicom_dir_sept,scanner_acq_date_sept,dicom_dir_check,acq_date_check,date_diff
0,MNI0028,ses-01,2023-10-04,MNI0028_152209_MRI01_MNI_20231004_152124681,MNI0028_152209_MRI01_MNI_20231004_152124681,2023-10-04,True,True,0 days
1,MNI0056,ses-01,2021-08-18,MNI0056_864854_MRI01_MNI_20210818_151510608,MNI0056_864854_MRI01_MNI_20210818_151510608,2021-08-18,True,True,0 days
2,MNI0058,ses-01,2021-08-18,MNI0058_197308_MRI01_MNI_20210818_105219098,MNI0058_197308_MRI01_MNI_20210818_105219098,2021-08-18,True,True,0 days
3,MNI0068,ses-01,2021-08-27,MNI0068_842090_MRI01_MNI_20210827_150412426,MNI0068_842090_MRI01_MNI_20210827_150412426,2021-08-27,True,True,0 days
4,MNI0079,ses-01,2021-12-22,MNI0079_760662_MRI01_MNI_20211222_134225878,MNI0079_760662_MRI01_MNI_20211222_134225878,2021-12-22,True,True,0 days


### Identify participants with different dicom directories or acquisition dates

In [8]:
mismatch_df = mri_acq_dates_df[~mri_acq_dates_df["dicom_dir_check"] | ~mri_acq_dates_df["acq_date_check"]]

mismatch_df

,participant_id,session,scanner_acq_date,participant_dicom_dir,participant_dicom_dir_sept,scanner_acq_date_sept,dicom_dir_check,acq_date_check,date_diff
60,MNI0369,ses-01,NaT,NaN,MNI0369,NaT,False,False,NaT
61,MNI0369,ses-02,NaT,NaN,MNI0369,NaT,False,False,NaT
76,MNI0414,ses-02,2024-03-28,MNI0414_201131_MNR02_MNI_20240328_154215338,MNI0414_201131_MNR02_MNI_20240328_154215338,NaT,True,False,NaT
87,MNI0436,ses-01,NaT,MNI0436,MNI0436,NaT,True,False,NaT
113,MNI0535,ses-01,2023-10-10,MNI0535_996589_MRI01_MNI_20231010_092131613,MNI0535_996589_MRI01_MNI_20231010_092131613,2023-10-06,True,False,4 days
173,PD00849,ses-03,2023-10-19,PD00849_337832_MRI03_MNI_20231019_171458528,PD00849_337832_MRI03_MNI_20231019_171458528,NaT,True,False,NaT
184,PD00955,ses-02,2022-01-28,PD00955_708031_MRI02_MNI_20220128_110518924,PD00955_708031_MRI02_MNI_20220128_110518924,NaT,True,False,NaT
202,PD01100,ses-01,NaT,PD01100,PD01100,NaT,True,False,NaT
222,PD01199,ses-03,2023-04-14,PD01199_120069_MRI03_MNI_20230414_180020217,PD01199_120069_MRI03_MNI_20230414_180020217,NaT,True,False,NaT
227,PD01223,ses-01,2019-01-10,PD01223_20220121_121713387,PD01223_20220121_121713387,NaT,True,False,NaT
